# Conversation Knowledge Graph
This type of memory uses a knowledge graph to recreate memory.

## Using memory with LLM

In [1]:
from langchain.memory import ConversationKGMemory
from langchain_openai import OpenAI

In [4]:
llm = OpenAI(temperature=0)
memory = ConversationKGMemory(llm=llm)
memory.save_context({"input": "say hi to sam"}, {"output": "who is sam"})
memory.save_context({"input": "sam is a friend"}, {"output": "okay"})

In [7]:
memory.load_memory_variables({"input": "who is sam"})

{'history': 'On sam: sam is a friend.'}

We can also get the history as a list of messages (this is useful if you are using this with a chat model).

In [8]:
memory = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context({"input": "say hi to sam"}, {"output": "who is sam"})
memory.save_context({"input": "sam is a friend"}, {"output": "okay"})

In [9]:
memory.load_memory_variables({"input": "who is sam"})

{'history': [SystemMessage(content='On sam: sam is a friend.')]}

In [10]:
memory.get_current_entities("what's Sams favorite color?")

['Sam']

We can also more modularly get knowledge triplets from a new message (will use previous messages as context).

In [12]:
memory.get_knowledge_triplets("her favorite color is red")

[KnowledgeTriple(subject='sam', predicate='is a', object_='friend'),
 KnowledgeTriple(subject='sam', predicate='favorite color', object_='red')]

## Using in a chain

In [19]:
llm = OpenAI(temperature=0)
from langchain.chains.conversation.base import ConversationChain
from langchain_core.prompts.prompt import PromptTemplate

template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(input_variables=["history", "input"], template=template)
conversation_with_kg = ConversationChain(
    llm=llm,
    # verbose=True,
    prompt=prompt, memory=ConversationKGMemory(llm=llm)
)

In [20]:
conversation_with_kg.predict(input="Hi, what's up?")

' Hello! I am an AI programmed to assist with various tasks and provide information. Currently, I am running on a server with 64 GB of RAM and a 2.5 GHz processor. I am also connected to a high-speed internet connection with a bandwidth of 1 Gbps. Is there something specific you would like to know?'

In [21]:
conversation_with_kg.predict(
    input="My name is James and I'm helping Will. He's an engineer."
)

" Hello James, it's nice to meet you. I am an AI designed to assist with various tasks. Will sounds like a very interesting person. As an engineer, he must have a strong understanding of math, science, and problem-solving. Is there anything specific you need help with?"

In [22]:
conversation_with_kg.predict(input="What do you know about Will?")

' Will is an engineer. He graduated from the University of California, Berkeley with a degree in mechanical engineering. He currently works at a tech company in Silicon Valley and specializes in designing and building robots for industrial use. He is also an avid hiker and enjoys spending his weekends exploring the nearby mountains and trails. Is there anything specific you would like to know about Will?'